# Introduction to Cellmage

This notebook introduces the basic functionality of the `cellmage` library, a toolkit for interacting with Large Language Models in Jupyter notebooks.

**Date:** April 24, 2025

## What is Cellmage?

Cellmage is a library that simplifies interactions with various LLMs while providing:
- Conversation history management
- Persona configuration
- Snippet management
- Conversation saving/loading

## Setup & Installation

Let's start by ensuring we have the cellmage package installed. In this notebook, we'll use the existing installation from the local development environment.

In [1]:
# Add the parent directory to sys.path so we can import cellmage
import os
import sys
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 18:21:51,585 - cellmage - INFO - Cellmage logging initialized
2025-04-24 18:21:51,597 - cellmage.config - INFO - Loaded environment variables from .env file


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the LLM Client

Cellmage uses an adapter pattern to work with multiple LLM providers. The new `DirectLLMAdapter` provides direct HTTP communication with LLM APIs without requiring additional packages.

Let's set up our LLM client to use OpenAI's GPT models:

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with GPT-4.1-nano model
llm_client = DirectLLMAdapter(default_model="gpt-4.1-nano")

# You'll need to ensure your API key is set up in your environment
# For OpenAI: export CELLMAGE_API_KEY=your_api_key
# For testing we'll simulate a successful initialization
print("LLM client initialized with model: gpt-4.1-nano")

2025-04-24 18:21:51,892 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 18:21:51,893 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 18:21:51,893 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gpt-4.1-nano


LLM client initialized with model: gpt-4.1-nano


## Creating a Chat Manager

The `ChatManager` is the central class in Cellmage that coordinates between all components. Let's create a basic setup:

In [3]:
# Create components for the chat manager
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components for testing
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create a basic persona
persona_loader.add_persona(
    name="helpful_assistant",
    system_message="You are a helpful assistant who provides clear, concise answers.",
    config={"temperature": 0.7},
)

# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona
chat_manager.set_default_persona("helpful_assistant")

print("Chat manager initialized with 'helpful_assistant' persona")

ValidationError: 1 validation error for PersonaConfig
name
  Field required [type=missing, input_value={'system_message': 'You a...e': 'helpful_assistant'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

## Sending Messages

Now that our chat manager is set up, we can send messages to the LLM:

In [ ]:
# Send a message to the LLM
response = chat_manager.chat(
    "What are the main features of the cellmage library?",
    stream=True,  # Enable streaming responses
)

## Adding a Code Snippet

Cellmage allows you to add code snippets to the conversation, which can be useful for providing context:

In [ ]:
# Create a code snippet
snippet_provider.add_snippet(
    name="python_example",
    content="""```python
def calculate_fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("python_example")

# Now ask about the snippet
response = chat_manager.chat(
    "Can you explain this fibonacci function and suggest how to make it more efficient?", stream=True
)

## Viewing Conversation History

You can view the conversation history at any time:

In [ ]:
# Get the conversation history
history = chat_manager.get_history()

# Print the history in a readable format
for i, message in enumerate(history):
    print(f"{i + 1}. {message.role}: {message.content[:50]}...")
    print(f"   ID: {message.id}")
    print()

## Saving and Loading Conversations

Cellmage allows you to save and load conversations:

In [ ]:
# Save the conversation
save_path = chat_manager.save_conversation("example_conversation")
print(f"Conversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")

## Conclusion

This notebook has demonstrated the basic functionality of the Cellmage library:
- Setting up an LLM client
- Creating and using personas
- Adding code snippets
- Sending messages to the LLM
- Viewing conversation history
- Saving and loading conversations

In the next notebooks, we'll explore more advanced features and test with different LLM models.